In [ ]:
%load_ext cython

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

The function `kernel(ξ, q)` below computes the integral kernel
$$K_q(\xi)=E_1\left(\frac{\xi^2}{2}\right)\frac{1}{|\Delta_q(\xi)|^2}\,\exp\left(-\frac{\xi^2}{2}\right),$$
where the dispersion function
$$\Delta_q(\xi)=1-q^2 W(\xi).$$
Here,
$$q=k_{\mathrm{J}}/k$$
is the dimensionless wavelength. Note that $\Delta_q(\xi)$ attains its vacuum value in the limit of infinitely short wavelength, i.e. $\Delta_0(\xi)=1$.

$W(\zeta)$ is the plasma dispersion function as defined by Ichimaru (1973). It is given by
$$W(\zeta)=\frac{1}{\sqrt{2\pi}}\int_{-\infty}^\infty\mathrm{d}x\,\frac{\exp(-x^2/2)}{x - \zeta}$$
for $\mathrm{Im}\,\zeta>0$ and through analytic continuation for $\mathrm{Im}\,\zeta\le 0$. In terms of the Faddeeva function (`scipy.special.wofz`)
$$w(\zeta)=\exp(-\zeta^2)\,\mathrm{erfc}(-i\zeta),$$
Ichmaru's plasma dispersion function is given by
$$W(\zeta)=1 + i\sqrt{\frac{\pi}{2}}\,\zeta\,w\left(\frac{\zeta}{\sqrt{2}}\right).$$

In [ ]:
from math import pi, sqrt, exp

def W(ζ):
    z = sqrt(0.5)*ζ
    return 1 + 1j*sqrt(pi)*z*sp.special.wofz(z)

def kernel(ξ, q):
    Δ = 1 - q*q*W(ξ)
    return exp(-0.5*ξ*ξ)*sp.special.exp1(0.5*ξ*ξ)/abs(Δ)**2

Below is a Cython implementation of the same function. This can be passed to `scipy.integrate.quad`, which then runs a lot faster. Note that on the real axis, the real and imaginary parts of $W(x)$ are given by
$$\mathrm{Re}\,W(\xi) = 1 - \sqrt{2}\,\xi\,F\left(\frac{\xi}{\sqrt{2}}\right)$$
and
$$\mathrm{Im}\,W(\xi) = i\sqrt{\frac{\pi}{2}}\,\xi\,\exp\left(-\frac{\xi^2}{2}\right),$$
where
$$F(x)=\exp(-x^2)\int_0^x\mathrm{d}t\,\exp(t^2)$$
is Dawson's function (`scipy.special.dawsn`).

In [ ]:
%%cython

from cpython.pycapsule cimport PyCapsule_New

import cython
from libc.math cimport pi, sqrt, exp
from scipy.special.cython_special cimport dawsn, exp1

@cython.cdivision(True)
cdef double c_kernel(int n, double[2] args):

    cdef double x, q, xi, F, W_r, W_i, D_r, D_i

    xi = args[0]
    q = args[1]

    x = sqrt(0.5)*xi

    W_r = 1 - 2*x*dawsn(x)
    W_i = sqrt(pi)*x*exp(-x*x)

    D_r = 1 - q*q*W_r
    D_i =   - q*q*W_i

    return exp(-x*x)*exp1(x*x)/(D_r*D_r + D_i*D_i)


def kernel_fast(xi, q):

    cdef double[2] args

    args[0] = xi
    args[1] = q

    return c_kernel(2, args)


def get_LowLevelCallable():

    from scipy import LowLevelCallable

    func_capsule = PyCapsule_New(<void*>c_kernel, "double (int, double *)", NULL)

    return LowLevelCallable(func_capsule)

Make sure the Cython version gives the same result as the pure Python version.

In [ ]:
for i in range(10):
    ξ = np.random.randn()
    q = np.random.rand()
    assert np.isclose(kernel(ξ, q), kernel_fast(ξ, q))

The following function computes the dimensionless relaxation rate
$$\frac{\Gamma}{k_{\mathrm{J}}\sigma\mu}=\sum_{\boldsymbol{n}\ne 0} q^3\int_0^\infty\mathrm{d}\xi\,K_q(\xi),$$
where the integral kernel $K_q(\xi)$ is defined above. The sum over wavenumbers $\boldsymbol{n}$ is computed as follows.

We suppose that the stellar system is contained inside a periodic cube of volume $L^3$. The wavevectors are then given by
$$\boldsymbol{k}=\frac{2\pi}{L}\boldsymbol{n}.$$
The magnitude of the wavevector, i.e. $k=|\boldsymbol{k}|$, is obviously
$$k=\frac{2\pi}{L}n.$$
Given this, the dimensionless wavelength $q=k_{\mathrm{J}}/k$ can be written as
$$q=\frac{L}{\lambda_{\mathrm{J}}}\frac{2\pi/L}{k}=\frac{q_0}{n},$$
where $\lambda_{\mathrm{J}}=2\pi/k_{\mathrm{J}}$ is the Jeans length. The above equation serves as the definition of the dimensionless box size $q_0=L/\lambda_{\mathrm{J}}$.

The function below sums over all wavenumbers $\boldsymbol{n}$ satisfying $0<n<5$. It computes both the *dressed* relaxation rate as defined above and the *vacuum* relaxation rate
$$\frac{\Gamma_0}{k_{\mathrm{J}}\sigma\mu}=\sum_{\boldsymbol{n}\ne 0} q^3\int_0^\infty\mathrm{d}\xi\,K_0(\xi),$$
which neglects self-gravity. The integral over $\xi$ can be taken out of the sum over $\boldsymbol{n}$.

In [ ]:
# Maximum magntitude of the wavenumber
nmax = 5

# Get a vector of squared wavenumbers |n|² that satisfy 0 < nx**2 + ny**2 + nz**2 <= nmax**2.
n2 = []
for nx in range(-nmax, nmax + 1):
    for ny in range(-nmax, nmax + 1):
        for nz in range(-nmax, nmax + 1):
            n2.append(nx*nx + ny*ny + nz*nz)
n2 = np.array(n2)
mask = np.logical_or(n2 == 0, n2 > nmax*nmax)
n2_compressed = np.ma.masked_where(mask, n2).compressed()

# Sort the squared wave numbers
n2_sorted = np.sort(n2_compressed)

# Inverse of the magnitude of the wavenumbers
ninv = 1/np.sqrt(n2_sorted)

# Array of dimensionless box sizes
q0 = np.linspace(0.01, 0.99, 128)

# Arrays to hold the dressed and bare relaxation rates
Γ = np.zeros_like(q0)
Γ0 = np.zeros_like(q0)

max_error = 0

# Get the fast, Cython-based integral kernel
cython_kernel = get_LowLevelCallable()

# Loop over box sizes
for j in range(q0.size):

    # Sum over wavenumbers
    for n1 in ninv:

        # Dimensionless inverse wavenumber
        q = q0[j]*n1

        # Short hand
        q3 = q**3

        # Dressed relaxation rate per wavenumber
        dressed, error = sp.integrate.quad(cython_kernel, 0.0, np.inf, args=(q,))
        max_error = max(error, max_error)
        Γ[j] += q3*dressed

        # Vacuum relaxation rate per wavenumber. First factor ...
        Γ0[j] += q3

    # ... and second factor
    bare, error = sp.integrate.quad(cython_kernel, 0.0, np.inf, args=(0,))
    max_error = max(error, max_error)
    Γ0[j] *= bare

    print("\r{:.0%}".format((j+1)/q0.size), end='', flush=True)

print("\nMaximum error: {}".format(max_error))

# Plot the ratio of dressed to vacuum relaxation rate
plt.semilogy(q0, Γ/Γ0 - 1);